## Problema do Projeto de Viga Soldada _(Welded Beam Design)_
### Um problema de Engenharia Estrutural

A engenharia estrutural, em sua essência, é a busca pelo equilíbrio entre segurança, funcionalidade e economia. O problema que estamos tratando foi formulado seminalmente por __K. M. Ragsdell e D. T. Phillips__ em 1976, através do artigo _"Optimal Design of a Class of Welded Structures Using Geometric Programming"_, publicado no _"Journal of Engineering for Industry"_. Eles propuseram a aplicação da Programação Geométrica _(Geometric Programming)_ para resolver problemas de design com restrições não-lineares complexas. O método busca minimizar o custo e modelar falhas físicas, transformando estes fenômenos em inequações algébricas tratáveis. Em seguida serão analiasadas cada variável, cada termo da função objetivo e a física subjacente a cada restrição. Mais detalhes podem ser obtidos na nossa apresentação, clicando aqui.

### Descrição do problema:

 Trata-se de uma viga rígida de aço, de seção transversal retangular, que deve ser fixada a um suporte rígido, como uma coluna ou parede; a fixação é realizada através de soldagem. A viga é projetada para suportar uma carga pontual $P$ aplicada em sua extremidade livre. A eficácia do conjunto depende de suas dimensões e da qualidade da solda. __O problema de otimização reside na determinação das dimensões exatas que minimizem o custo de fabricação sem que a estrutura falhe__. Devido à natureza altamente não-linear das restrições do problema e à complexidade da superfície de busca, o problema da viga soldada tornou-se um "padrão-ouro" (benchmark) para testar a eficácia de novos algoritmos de otimização.

### Variáveis (Vetor de Projeto)

Este problema é definido por quatro variáveis contínuas. Para este relatório, adotaremos a notação vetorial $${x} = [x_1, x_2, x_3, x_4]^T$$ 
| Variável | Descrição | 
|:---------|:------:|
| $x_1$ |   __Espessura da solda:__ Dimensão das pernas do filete de solda. É a dimensão crítica que define a área resistente ao cisalhamento     |
| $x_2$       |   __Comprimento da solda:__ O comprimento onde a viga toca o suporte, onde a solda é aplicada. Afeta tanto a resistência da solda quanto o braço de alavanca   | 
| $x_3$       |    __Altura da viga:__ Dimensão vertical da viga. Variável mais influente na resistência à deflexão    |
| $x_4$       |   __Largura da viga:__ A dimensão horizontal da viga. Fundamental para a estabilidade lateral (flambagem)    |    


### Definição do Espaço de Busca

Usaremos os limites padrão utilizados na maioria dos estudos contemporâneos (e.g., bibliotecas como _NEORL_ ou _MATLAB GADS_): $x_1, x_4 \in [0.1, 2.0]$ e $x_2, x_3 \in [0.1, 10.0]$

| Variável | Limite Superior | Limite Inferior |
|:---------|:------|:------:|
| $x_1$ |   0.1    |   2.0    |
| $x_2$ |   0.1    |   10.0   | 
| $x_3$ |   0.1    |   10.0   | 
| $x_4$ |   0.1    |   2.0    |

### Função objetivo

O objetivo central do problema é econômico: minimizar o custo total de fabricação. Ragsdell e Phillips modelaram este custo como a soma de dois componentes principais: o custo do material da viga e o custo da mão-de-obra e material da soldagem. A função objetivo $f({x})$ é expressa como: $f({x}) = C_{solda} + C_{viga}$

Expandindo os termos, temos $f({x}) = (c_1 + c_0) \cdot V_{solda} + c_2 \cdot V_{viga}$, onde:

$V_{solda} \approx x_1^2 \cdot x_2$ 

$V_{viga} = x_3 \cdot x_4 \cdot (L + x_2)$ 

$c_0$: Custo de instalação 

$c_1$: Custo pela solda (material + mão-de-obra)

$c_2$: Custo pela barra de aço

Expandindo novamente, temos:
$ (c_1 + c_0) \cdot (x_1^2 \cdot x_2) + c_2 \cdot (x_3 \cdot x_4 \cdot (L + x_2)) $

Na formulação padrão, os custos unitários são coeficientes numéricos fixos, resultando na __equação canônica:__ $$f({x}) = 1.10471 x_1^2 x_2 + 0.04811 x_3 x_4 (14.0 + x_2)$$

__Nota:__ estes coeficientes numéricos foram encontrados através de uma pesquisa de mercado realizada por Ragsdell e Phillips na década de 70. 

#### Trade-offs: 
- __O Alto Custo da Soldagem ($1.10471$):__ Reflete a realidade industrial onde a mão-de-obra especializada e os consumíveis de soldagem são significativamente caros.
__Implicação: O otimizador será pressionado a reduzir $x_1$ (espessura) e $x_2$ (comprimento) para baixar o custo relacionado à solda, trazendo um risco: comprometer a resistência ao cisalhamento (restrição $g_1$)__
- __A Dependência Quadrática ($x_1^2$):__ O custo da solda cresce quadraticamente com a espessura ($x_1$). Dobrar a espessura da solda quadruplica seu volume e custo.
__Implicação: É economicamente preferível ter uma solda mais longa ($x_2$) e fina ($x_1$) do que uma solda curta e grossa, desde que a geometria permita__
- __O Custo do Material ($x_3, x_4$):__ O custo da viga é linear em relação às suas dimensões.
__Implicação: Aumentar a altura da viga ($x_3$) aumenta o custo linearmente, mas aumenta a inércia (resistência à flexão) cubicamente, o que favorece vigas altas e finas__

### Modelagem das restrições:

As restrições $g_j({x}) \leq 0$ representam modos de falha física. A violação de uma restrição implica o colapso da estrutura. Existem sete restrições principais no modelo padrão. O completo entendimento das restrições implica conhecimento profundo em engenharia e física; para este projeto, vamos considerá-las como "verdades universais"

#### Constantes Físicas:
- Carga Aplicada ($P$): 6.000 libras-força
- Comprimento do Balanço ($L$): 14 polegadas
- Módulo de Elasticidade ($E$): $30 \times 10^6$ Libras por polegada quadrada (psi)
- Módulo de Cisalhamento ($G$): $12 \times 10^6$ psi 
- Tensão Máxima de Cisalhamento ($\tau_{max}$): 13.600 psi
- Tensão Máxima de Flexão ($\sigma_{max}$): 30.000 psi
- Deflexão Máxima Permitida ($\delta_{max}$): 0.25 polegadas

#### Restrições:

- __Restrição $g_1$: Tensão de Cisalhamento na Solda__

__Tensão de Cisalhamento Primária ($\tau'$):__  $$\tau' = \frac{P}{\sqrt{2} x_1 x_2}$$
__Tensão de Cisalhamento Secundária ($\tau''$):__ $$\tau'' = \frac{M \cdot R}{J}$$
Momento ($M$): $$M = P \cdot (L + \frac{x_2}{2})$$
Raio Polar ($R$): $$R = \sqrt{ \left(\frac{x_2}{2}\right)^2 + \left(\frac{x_1 + x_3}{2}\right)^2 }$$
Momento Polar de Inércia ($J$): $$J = 2 \left\{ \sqrt{2} x_1 x_2 \left[ \frac{x_2^2}{12} + \left(\frac{x_1 + x_3}{2}\right)^2 \right] \right\}$$
__Tensão Combinada ($\tau$):__ $$\tau({x}) = \sqrt{ (\tau')^2 + (\tau'')^2 + 2 \tau' \tau'' \frac{x_2}{2R} }$$

__A Inequação ($g_1$):__ $\tau({x}) - \tau_{max} \leq 0$

- __Restrição $g_2$: Tensão de Flexão na Viga__
Momento Fletor Máximo: $$M_{viga} = P \cdot L = 6000 \cdot 14$$
Módulo de Resistência ($S$): $$S = \frac{x_4 x_3^2}{6}$$
Tensão Normal ($\sigma$): $$\sigma({x}) = \frac{6 P L}{x_4 x_3^2}   ou   \frac{504.000}{x_4 x_3^2}$$

__A Inequação ($g_2$):__ $g_2({x}) = \sigma({x}) - \sigma_{max} \leq 0$

- __Restrição $g_3$ - Compatibilidade Geométrica Solda-Viga:__
Uma restrição prática de fabricação: a espessura do cordão de solda ($x_1$) não pode ser maior que a largura da viga ($x_4$) onde ela é aplicada.

__A Inequação ($g_3$):__ $g_3({x}) = x_1 - x_4 \leq 0$

- __Restrição $g_4$ - Restrição de Custo:__
Esta restrição força o algoritmo a encontrar soluções abaixo de um valor arbitrário, que representa unidades monetárias, criando uma região viável artificialmente limitada pelo orçamento. Frequentemente este valor é associado a 5.0

__A Inequação ($g_4$):__ $g_4({x}) = 1.10471 x_1^2 x_2 + 0.04811 x_3 x_4 (14.0 + x_2) - 5.0 \leq 0$

- __Restrição $g_5$ - Tamanho Mínimo da Solda:__
Para garantir a integridade da solda, impõe-se um limite inferior. Embora o espaço de busca já defina $x_1 \geq 0.1$, esta restrição é frequentemente adicionada para forçar um valor mínimo viável, como 0.125 (1/8 polegada)

__A Inequação ($g_5$):__ $g_5({x}) = 0.125 - x_1 \leq 0$

- __Restrição $g_6$ - Deflexão da Ponta:__
Para garantir a utilidade da estrutura, a viga não deve se deformar excessivamente. Caso ocorra, pode inutilizar o equipamento $$\delta({x}) = \frac{4 P L^3}{E x_4 x_3^3} ou \frac{65.856.000}{30 \times 10^6 x_4 x_3^3}$$

__A Inequação ($g_6$):__ $g_6({x}) = \delta({x}) - \delta_{max} \leq 0$

- __Restrição $g_7$ - Flambagem Lateral-Torsional:__
Ragsdell e Phillips utilizaram uma aproximação para a carga crítica de flambagem ($P_c$) derivada da teoria de estabilidade elástica de Timoshenko $$P_c({x}) = \frac{4.013 E \cdot x_3 x_4^3}{6 L^2} \left( 1 - \frac{x_3}{2L} \sqrt{\frac{E}{4G}} \right)$$
Numericamente, inserindo as constantes: $$P_c({x}) = 64746.022 \cdot (1 - 0.0282346 \cdot x_3) \cdot x_3 \cdot x_4^3$$

__A Inequação ($g_7$):__ $6000 - P_c({x}) \leq 0$

Esta restrição é o "antídoto" para a restrição de flexão ($g_2$). Enquanto $g_2$ pede uma viga alta e fina, $g_7$ pune severamente vigas finas. O ótimo global do problema reside, portanto, no limite preciso onde a viga é alta o suficiente para não falhar por deflexão, mas larga o suficiente para não flambar.


## Implementação do código
#### Chegamos na parte prática do projeto, onde vamos implementar dois métodos diferentes para resolver o problema formulado e descrito acima. Para fins de comparação e correção, buscamos o valor de consenso para o ótimo global "verdadeiro": __2.3809 a 2.3810__ (sob as restrições estritas de Ragsdell)


links:
https://doi.org/10.1115/1.3438995

https://doi.org/10.2514/3.10834

https://doi.org/10.1016/S0166-3615(99)00046-9

https://doi.org/10.1016/j.cma.2004.09.007

https://doi.org/10.1109/TEVC.2003.814902